# Part 2: Multiple Knowledge Sources

In Part 1, you queried a single knowledge source. In part 2, you'll scale up to query multiple knowledge sources (HR + Health docs) from a single knowledge base. You'll learn how to control which sources get queried and how to guide the agentic behavior of the Knowledge Base using natural language instructions.

## Step 1: Load Environment Variables

Run below cell to load the configuration for your Azure resources, choose the **.venv(3.11.9)** environment that is created for you. 

Note that this time, the knowledge base name reflects that we're working with multiple sources: `hr-and-health-docs-knowledge-base`.

> **⚠️ Troubleshooting**
>
> If code cells get stuck and keep spinning, select **Restart** from the notebook toolbar at the top. If the issue persists after a couple of tries, close VS Code completely and reopen it.

In [1]:
import os

from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv

load_dotenv(override=True) # take environment variables from .env.

# Azure AI Search configuration
endpoint = os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"]
credential = AzureKeyCredential(os.environ["AZURE_SEARCH_ADMIN_KEY"])

# Knowledge base name
knowledge_base_name = "hr-and-health-docs-knowledge-base"

# Azure OpenAI configuration
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_key = os.environ["AZURE_OPENAI_KEY"]
azure_openai_chatgpt_deployment = os.getenv("AZURE_OPENAI_CHATGPT_DEPLOYMENT", "gpt-4.1")
azure_openai_chatgpt_model_name = os.getenv("AZURE_OPENAI_CHATGPT_MODEL_NAME", "gpt-4.1")

print("Environment variables loaded")

Environment variables loaded


## Step 2: Create Two Knowledge Sources

You'll create two knowledge sources:
- **healthdocs-knowledge-source**: Points to the `healthdocs` index (334 document chunks about health benefits and insurance)
- **hrdocs-knowledge-source**: Points to the `hrdocs` index (50 document chunks about HR policies)

Both sources connect to different indexes but use the same field configuration (`blob_path` for citations, `snippet` for content).

By creating multiple knowledge sources, you enable the knowledge base to intelligently decide which data to query based on the user's question.

In [2]:
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndexFieldReference, SearchIndexKnowledgeSource, SearchIndexKnowledgeSourceParameters

index_client = SearchIndexClient(endpoint=endpoint, credential=credential)

ks = SearchIndexKnowledgeSource(
    name="healthdocs-knowledge-source",
    description="Zava health documents: health benefits and insurance plan information",
    search_index_parameters=SearchIndexKnowledgeSourceParameters(
        search_index_name="healthdocs",
        source_data_fields=[SearchIndexFieldReference(name="blob_path"), SearchIndexFieldReference(name="snippet")],
        search_fields=[SearchIndexFieldReference(name="snippet")]
    ),
)
index_client.create_or_update_knowledge_source(knowledge_source=ks)
print(f"Knowledge source '{ks.name}' created or updated successfully.")

ks = SearchIndexKnowledgeSource(
    name="hrdocs-knowledge-source",
    description="Zava HR documents: company policies, job roles, workplace guidelines, and general benefits",
    search_index_parameters=SearchIndexKnowledgeSourceParameters(
        search_index_name="hrdocs",
        source_data_fields=[SearchIndexFieldReference(name="blob_path"), SearchIndexFieldReference(name="snippet")],
        search_fields=[SearchIndexFieldReference(name="snippet")]
    )
)

index_client.create_or_update_knowledge_source(knowledge_source=ks)
print(f"Knowledge source '{ks.name}' created or updated successfully.")

Knowledge source 'healthdocs-knowledge-source' created or updated successfully.
Knowledge source 'hrdocs-knowledge-source' created or updated successfully.


## Step 3: Create Combined Knowledge Base

Now create a knowledge base that references both knowledge sources. Notice how the `knowledge_sources` parameter takes a list of references.

This single knowledge base can now query both HR and Health documents. When you send a query, the knowledge base will analyze the question and determine which sources are relevant and it can query one source, both sources, or intelligently select based on the question content.

In [3]:
from azure.search.documents.indexes.models import AzureOpenAIVectorizerParameters, KnowledgeBase, KnowledgeBaseAzureOpenAIModel, KnowledgeRetrievalOutputMode, KnowledgeSourceReference

aoai_params = AzureOpenAIVectorizerParameters(
    resource_url=azure_openai_endpoint,
    deployment_name=azure_openai_chatgpt_deployment,
    model_name=azure_openai_chatgpt_model_name,
    api_key=azure_openai_key
)

knowledge_base = KnowledgeBase(
    name=knowledge_base_name,
    models=[KnowledgeBaseAzureOpenAIModel(azure_open_ai_parameters=aoai_params)],
    knowledge_sources=[
        KnowledgeSourceReference(name="healthdocs-knowledge-source"),
        KnowledgeSourceReference(name="hrdocs-knowledge-source")
    ],
    output_mode=KnowledgeRetrievalOutputMode.ANSWER_SYNTHESIS
)

index_client.create_or_update_knowledge_base(knowledge_base)
print(f"Knowledge base '{knowledge_base_name}' created or updated successfully.")

Knowledge base 'hr-and-health-docs-knowledge-base' created or updated successfully.


## Step 4: Query Multiple Sources

Let's ask two questions in one query:
- "What is the responsibility of the Zava CEO?" (HR-related)
- "What health plan would you recommend if they wanted the best coverage for mental health services?" (Health-related)

When you run this query, the knowledge base uses agentic retrieval:
1. Analyzes the query to understand you're asking about two different topics
2. Decomposes the query into focused subqueries (one for each sub-topic in the query)
3. Determines which knowledge sources are relevant for each subquery
4. Runs searches concurrently against the selected sources
5. Uses semantic ranker to rerank and filter results
6. Synthesizes a coherent answer from both sources

By default, the knowledge base intelligently selects which sources to query. It might only query the sources it deems relevant, which you'll verify in the next step.

In [4]:
from azure.search.documents.knowledgebases import KnowledgeBaseRetrievalClient
from azure.search.documents.knowledgebases.models import KnowledgeBaseMessage, KnowledgeBaseMessageTextContent, KnowledgeBaseRetrievalRequest, SearchIndexKnowledgeSourceParams
from IPython.display import display, Markdown

knowledge_base_client = KnowledgeBaseRetrievalClient(endpoint=endpoint, knowledge_base_name=knowledge_base_name, credential=credential)

healthdocs_ks_params = SearchIndexKnowledgeSourceParams(
    knowledge_source_name="healthdocs-knowledge-source",
    include_references=True,
    include_reference_source_data=True
)
hrdocs_ks_params = SearchIndexKnowledgeSourceParams(
    knowledge_source_name="hrdocs-knowledge-source",
    include_references=True,
    include_reference_source_data=True
)
req = KnowledgeBaseRetrievalRequest(
    messages=[
        KnowledgeBaseMessage(role="user", content=[KnowledgeBaseMessageTextContent(text="""
            What is the responsibility of the Zava CEO?
            What health plan would you recommend if they wanted the best coverage for mental health services?
        """)])
    ],
    knowledge_source_params=[
        healthdocs_ks_params,
        hrdocs_ks_params
    ],
    include_activity=True
)

result = knowledge_base_client.retrieve(retrieval_request=req)
display(Markdown(result.response[0].content[0].text))

The responsibilities of the Zava CEO include providing strategic direction and oversight to ensure the company's long-term success and profitability. Key duties involve developing and implementing company strategy and objectives, guiding the executive team, managing daily operations, ensuring legal compliance, overseeing marketing strategies, managing stakeholder relationships, representing the company publicly, negotiating contracts, monitoring industry trends, and fostering a positive company culture. The CEO should have a bachelor's degree in business or a related field, over 10 years of executive management experience, strong leadership and analytical skills, and knowledge of business finances and operations [ref_id:0].

For mental health coverage, the Northwind Health Plus plan offers the most comprehensive benefits, including coverage for counseling, psychiatric visits, therapy, group therapy, and both inpatient and outpatient services, with access to in-network and out-of-network providers. This makes it the best option for mental health services for the Zava CEO [ref_id:4][ref_id:8].

## Step 5: Map Citations to Knowledge Sources

This helper function traces each citation in the answer back to its original knowledge source. It shows you whether information came from `hrdocs` or `healthdocs`.

You can use this function to see which sources the knowledge base chose, verify information origins, and check if it's picking the right sources for different question types.

Run the cell below to see which knowledge source each citation came from.

In [5]:
import re


def find_source_of_reference(reference_id):
    activity_id = None
    for reference in result.references:
        if reference.id == reference_id:
            activity_id = reference.activity_source
            break
    for activity in result.activity:
        if activity.id == activity_id:
            return activity.knowledge_source_name
    return None

def cite_sources(text):
    references = re.findall(r'\[ref_id:(\d+)\]', text)
    sources = {}
    for ref_id in references:
        source_info = find_source_of_reference(ref_id)
        if source_info:
            sources[ref_id] = source_info
    
    return sources

sources = cite_sources(result.response[0].content[0].text)
print("Cited sources:", sources)


Cited sources: {'0': 'hrdocs-knowledge-source', '4': 'healthdocs-knowledge-source', '8': 'healthdocs-knowledge-source'}


## Step 6: Force Querying All Sources

By setting `always_query_source=True` on both knowledge source parameters, you can force the knowledge base to query both sources regardless of whether it thinks they're relevant.

The code below runs a single simpler question. This time, both sources are queried because `always_query_source=True` overrides the intelligent selection. You'll see citations from both sources even when one wasn't strictly necessary, more comprehensive answers pulling from both sources, and different source mappings in the output.

This is useful when you want comprehensive coverage across all your data, but it comes at the cost of higher latency and token usage.

Try it with different questions and see how the results vary!

In [6]:
healthdocs_ks_params.always_query_source = True
hrdocs_ks_params.always_query_source = True

req = KnowledgeBaseRetrievalRequest(
    messages=[
        KnowledgeBaseMessage(role="user", content=[KnowledgeBaseMessageTextContent(text="What health benefits are there?")])
    ],
    knowledge_source_params=[
        healthdocs_ks_params,
        hrdocs_ks_params
    ],
    include_activity=True
)

result = knowledge_base_client.retrieve(retrieval_request=req)
display(Markdown(result.response[0].content[0].text))

sources_cited = cite_sources(result.response[0].content[0].text)
print("Cited sources:", sources)

sources_queried = [activity.knowledge_source_name for activity in result.activity if activity.type == "searchIndex"]
print("Queried sources:", sources_queried)

Health benefits available include coverage for preventive care services such as routine physicals, vaccinations (including flu, shingles, measles, mumps, and rubella), and screenings for conditions like cancer, diabetes, and high blood pressure. These services are covered at no additional cost and are designed to help detect health issues early and keep you healthy [ref_id:0][ref_id:1][ref_id:2][ref_id:3][ref_id:6].

Additional benefits include coverage for professional visits and services, such as visits to primary care physicians, specialists, diagnostic tests, physical therapy, mental health services, prescription drugs, inpatient hospital services, and emergency services, provided they are medically necessary and performed by in-network providers [ref_id:3].

There are also personal health support programs, which offer access to care coordinators, health coaches, care managers, health educators, and specialists for nutrition and lifestyle counseling. These programs help manage chronic illnesses, injuries, and other health-related issues [ref_id:8][ref_id:9].

For those with chronic conditions, a Chronic Condition Management Program is available, providing comprehensive care assessments, regular follow-up visits, personalized health education, medication management assistance, and coordination with other providers. This program is available for conditions such as diabetes, asthma, heart failure, coronary artery disease, COPD, chronic kidney disease, and hypertension [ref_id:4][ref_id:7][ref_id:9].

Some exceptions apply, such as no coverage for cosmetic services, experimental treatments, most dental services, and services provided by out-of-network providers [ref_id:3].

Cited sources: {'0': 'hrdocs-knowledge-source', '4': 'healthdocs-knowledge-source', '8': 'healthdocs-knowledge-source'}
Queried sources: ['healthdocs-knowledge-source', 'hrdocs-knowledge-source']


## Step 7: Guide Source Selection with Retrieval Instructions

**Retrieval instructions** let you guide the knowledge base's source selection using natural language instead of forcing it to query everything.

You can set `retrieval_instructions` to provide guidance like "Use healthdocs for health questions, hrdocs for HR questions." The knowledge base still makes the final decision, but now with your guidance. It can skip irrelevant sources for better efficiency and uses the context you provide to understand intent better than with `always_query_source=True`.

The first cell below updates the knowledge base with retrieval instructions. 

The second cell queries with `always_query_source=False` to see how the instructions guide source selection.

In [7]:
knowledge_base.retrieval_instructions="If the question is about health benefits or insurance specifically, use healthdocs. Otherwise, use hrdocs for other HR-related questions."

index_client.create_or_update_knowledge_base(knowledge_base)
print(f"Knowledge base '{knowledge_base_name}' created or updated successfully.")

Knowledge base 'hr-and-health-docs-knowledge-base' created or updated successfully.


In [8]:
healthdocs_ks_params.always_query_source = False
hrdocs_ks_params.always_query_source = False

req = KnowledgeBaseRetrievalRequest(
    messages=[
        KnowledgeBaseMessage(role="user", content=[KnowledgeBaseMessageTextContent(text="What health benefits are there?")])
    ],
    knowledge_source_params=[
        healthdocs_ks_params,
        hrdocs_ks_params
    ],
    include_activity=True
)


result = knowledge_base_client.retrieve(retrieval_request=req)
display(Markdown(result.response[0].content[0].text))

sources_cited = cite_sources(result.response[0].content[0].text)
print("Cited sources:", sources)

sources_queried = [activity.knowledge_source_name for activity in result.activity if activity.type == "searchIndex"]
print("Queried sources:", sources_queried)

A variety of health benefits are available, depending on the plan. Northwind Health Plus offers comprehensive coverage for medical, vision, and dental services, as well as prescription drugs, mental health and substance abuse services, preventive care, and emergency services (both in-network and out-of-network). You can choose from a range of in-network providers, including primary care physicians, specialists, hospitals, and pharmacies. The plan also covers services like dialysis and habilitation therapy for neurodevelopmental needs, and provides access to independent healthcare contractors if medically necessary. Preventive care services are covered without cost sharing, and the plan complies with relevant laws such as the Affordable Care Act and Mental Health Parity and Addiction Equity Act [ref_id:1][ref_id:2][ref_id:3][ref_id:4][ref_id:6][ref_id:7][ref_id:10].

Northwind Standard is a more basic plan, covering medical, vision, and dental services, preventive care, and prescription drugs, with access to in-network providers. However, it does not cover emergency services, mental health and substance abuse services, or out-of-network care [ref_id:2][ref_id:8][ref_id:9].

Both plans encourage the use of preventive care, in-network providers, and generic prescription drugs to help manage costs and maximize benefits [ref_id:1][ref_id:9][ref_id:10].

Cited sources: {'0': 'hrdocs-knowledge-source', '4': 'healthdocs-knowledge-source', '8': 'healthdocs-knowledge-source'}
Queried sources: ['healthdocs-knowledge-source', 'healthdocs-knowledge-source']


## Step 8: Format Answers with Answer Instructions

**Answer instructions** control how the knowledge base formats its response. They don't change which sources are queried or what information is retrieved - just how it's presented.

The code below uses `answer_instructions` to instruct the knowledge base to format answers as bulleted lists. You'll want to customize this setting whenever you need application-specific formatting.

The first cell below updates the knowledge base with the new `answer_instructions`. The second cell runs the same query to show the formatted results.

In [9]:
knowledge_base.answer_instructions="Always use a bulleted list format when providing answers. Each bullet should be on a separate line"

index_client.create_or_update_knowledge_base(knowledge_base)
print(f"Knowledge base '{knowledge_base_name}' created or updated successfully.")


Knowledge base 'hr-and-health-docs-knowledge-base' created or updated successfully.


In [10]:
result = knowledge_base_client.retrieve(retrieval_request=req)
display(Markdown(result.response[0].content[0].text))

A variety of health benefits are available, depending on the plan. The Northwind Health Plus plan offers comprehensive coverage for medical, vision, and dental services, as well as prescription drug coverage, mental health and substance abuse services, preventive care, and emergency services (both in-network and out-of-network) [ref_id:4][ref_id:5]. Members can choose from a range of in-network providers, including primary care physicians, specialists, hospitals, and pharmacies [ref_id:4][ref_id:5]. Preventive care services, such as check-ups and screenings, are covered at no additional cost [ref_id:3][ref_id:4][ref_id:5].

Additional benefits include personal health support programs like case management, disease management for chronic conditions, and wellness programs to help with smoking cessation, stress management, and overall well-being [ref_id:7]. The plan also covers services provided by independent contractors if they are medically necessary [ref_id:6].

For the Northwind Standard plan, coverage includes medical, vision, and dental services, preventive care, and prescription drugs, but does not cover emergency care, mental health and substance abuse services, or out-of-network services [ref_id:2][ref_id:8][ref_id:10].

It is important to review your specific plan documents to understand the exact benefits, coverage limits, exclusions, and cost-sharing requirements [ref_id:5][ref_id:9].

## Summary

You've now worked with multiple knowledge sources in a single knowledge base and learned how to control which ones get queried.

**Key concepts to remember:**
- A single knowledge base can reference multiple knowledge sources
- The knowledge base intelligently selects which sources to query based on the question
- `always_query_source=True` forces querying all sources
- `retrieval_instructions` guide source selection with natural language
- `answer_instructions` control response formatting without changing content

### What's Next?

➡️ Continue to [Part 3: SharePoint Knowledge Source](part3-sharepoint-knowledge-source.ipynb) to learn how to connect your knowledge base directly to SharePoint document libraries.